### Annotation
Annotations in Java serve multiple purposes. They can be used to provide extra information or for compile-time/runtime processing.

### Creating annotations
To create annotation:

In [1]:
public @interface Information{
    // String value();
    String author();
    String date();
    
    // optional annotation metadata
    int minVersion() default 1;
    String[] reviewers() default {};
}

`Class`, `String`, enums, primitives and arrays are the allowed annotation parameter types. `value` is a special parameter. When annotation is used as `@Information("Some value")` the argument is passed to value. In the above example, we don't use value, thereby force the programmer to pass author and date. 

### Annotations of Annotation
`@Documented` annotation can be used to indicate that Javadocs for the class, etc should contain this annotation.

What the annotation can be used with can be specified using `@Target` annotation. Possible values:
- `ElementType.TYPE` : Class, Interface or Enum
- `ElementType.FIELD`
- `ElementType.METHOD`
- `ElementType.PARAMETER` : Method parameter
- `ElementType.CONSTRUCTOR`
- `ElementType.LOCAL_VARIABLE`
- `ElementType.ANNOTATION_TYPE` : Another annotation
- `ElementType.PACKAGE`

For the above Information annotation example, we specify the following target:

In [ ]:
@Target(value = {ElementType.TYPE, ElementType.ANNOTATION_TYPE})
public @interface Information {
    // ...
}

How long the annotation should be kept is determined by `@Retention` annotation. Possible values are:
- `RetentionPolicy.SOURCE` : Discard during the compile. These annotations don't make any sense after the compile has completed, so they aren't written to the bytecode. Example: `@Override`, `@SuppressWarnings`
- `RetentionPolicy.CLASS` : Discard during class load. Useful when doing bytecode-level post-processing. Somewhat surprisingly, this is the default.
- `RetentionPolicy.RUNTIME` : Do not discard. The annotation should be available for reflection at runtime. Example: `@Deprecated`

### Annotation and Reflection
Annotations with retention policy as runtime can be extracted and its properties read at runtime. For example, if we change the retention policy of Information annotation to runtime,

In [ ]:
public class AnnotationDemo {
    public static void main(String[] args) {
        // Get all annotations directly applieed to the given element
        Annotation[] annotations = Demo.class.getDeclaredAnnotations();
        for (Annotation annotation : annotations) {
            if (annotation instanceof Information) {
                System.out.println("Author: " + ((Information) annotation).author());
                System.out.println("Date: " + ((Information) annotation).date());
            }
        }
    }
}

@Information(author = "John Doe", date = "12-12-2020")
class Demo {}

The above code can be shortened to:

In [ ]:
public static void main(String[] args) {
    if(Demo.class.isAnnotationPresent(Information.class)){
        Information info = Demo.class.getAnnotation(Information.class);
        System.out.println("Author: " + info.author());
        System.out.println("Date: " + info.date());
    }
}